<h1>Segmenting and Clustering Neighborhoods in North York</h1>
<h3>IBM Data Science Professional Certificate: Course 9 Final Assignment</h3>

Install and import the required libraries:

In [4]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

import requests
import json
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes
import geopy.geocoders
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium

from sklearn.cluster import KMeans

import pickle

Solving environment: ...working... done

# All requested packages already installed.



WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


Solving environment: ...working... done

# All requested packages already installed.



WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


Extract postal codes page information from Wikipedia: 

In [5]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(wiki_url).text
##print(page)

Trim 'Wikipedia page' into 'info' for further process:

In [6]:
#define function to look for <tbody>
def findContent1(node, page):    
    startIndex = page.find("<"+node+">")
    endIndex = page.find("</"+node+">")
    info = page[startIndex + len("<"+node+">"):endIndex]
    return info

#trim page
info = findContent1("tbody", page)
##print(info)

Transform info into dataframe:

In [7]:
#separate info in lines
data = info.split('\n')
##print(data)

#define two find content functions
def findContent2(node, page):    
    startIndex = page.find("<"+node+">")
    endIndex = page.find("</"+node+">")
    title = page[startIndex + len("<"+node+">"):endIndex]
    return title
def findContent3(node, page):
    startIndex = page.find(node)
    endIndex = page.find("</a"+node)
    title = page[startIndex + len(node):endIndex]
    return title

In [8]:
#initialization
postalcode = []
borough = []
neighborhood = []
l = len(data)

#search info
for i in range(6,l,5):
    t1 = findContent2("td", data[i + 1])
    if t1 != "Not assigned":
        t2 = findContent3(">", t1)
        borough.append(t2)
        postalcode.append(findContent2("td", data[i]))
        if data[i + 2][4:] != "Not assigned":
            t3 = findContent3(">", data[i + 2][4:])
            neighborhood.append(t3)
        else:
            if t1 != "Not assigned":
                neighborhood.append(t2)
            else:
                neighborhood.append(data[i + 2][4:])

In [9]:
#create dataframe using Pandas
d = {'PostalCode': postalcode, 'Borough': borough, 'Neighborhood': neighborhood}
df = pd.DataFrame(data=d)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [10]:
#check dataframe shape
df.shape

(211, 3)

Extract geographical coordinates and add latitude and longitude to dataframe:

In [11]:
#initialization
df['Latitude']=''
df['Longitude']=''

In [12]:
#read geographical coordinates file
df_cord = pd.read_csv('https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv')

In [13]:
#assign coordinates to dataframe
for i in range(len(df_cord)):
    df.at[df['PostalCode']==df_cord.iloc[i]['Postal Code'],'Latitude']=df_cord.iloc[i]['Latitude']
    df.at[df['PostalCode']==df_cord.iloc[i]['Postal Code'],'Longitude']=df_cord.iloc[i]['Longitude']

In [14]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
3,M5A,Downtown Toronto,Regent Park,43.6543,-79.3606
4,M6A,North York,Lawrence Heights,43.7185,-79.4648


Trim dataframe to contain info on North York only:

In [17]:
# selecting only neighborhoods regarding to "Scarborough" borough.
northyork_data = df[df['Borough'] == 'North York']
northyork_data = northyork_data.reset_index(drop = True)
northyork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M6A,North York,Lawrence Heights,43.7185,-79.4648
3,M6A,North York,Lawrence Manor,43.7185,-79.4648
4,M3B,North York,Don Mills Nort,43.7459,-79.3522


In [18]:
northyork_data.shape

(38, 5)

Plot North York on map using Folium:

In [19]:
#obtain geographical coordinates of North York
address = 'North York, Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of North York are {}, {}.'.format(latitude, longitude))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinates of North York are 43.7708175, -79.4132998.


In [20]:
#create map of North York
northyork_map = folium.Map(location=[latitude, longitude], zoom_start=10)
northyork_map

In [21]:
# add markers of neighborhood to map
for lat, lng, borough, neighborhood in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Borough'], northyork_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(northyork_map)  
northyork_map

FourSquare credentials:

In [22]:
CLIENT_ID = 'ZKUJHYWUCKLLXQ5LEWDW3DSNCVRST5CMX2WBAQUHKSJXLVPL'
CLIENT_SECRET = 'RBEZN211RQVMBJGTFFGHLCARS3WQBLGCSOGM3EPUE0IPJGKT'
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZKUJHYWUCKLLXQ5LEWDW3DSNCVRST5CMX2WBAQUHKSJXLVPL
CLIENT_SECRET:RBEZN211RQVMBJGTFFGHLCARS3WQBLGCSOGM3EPUE0IPJGKT


Define function to obtain info from FourSquare:

In [23]:
def fs_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    
    #initialization
    result_ds = []
    counter = 0
    
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        #create API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        #make GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; 
        tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; 
        tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
    
    return result_ds;

#execute the above function to obtain info from FourSquare into 
northyork_fsinfo = fs_crawler(list(northyork_data['PostalCode']), list(northyork_data['Neighborhood']), list(northyork_data['Latitude']), list(northyork_data['Longitude']),)
##print(northyork_fsinfo)
##len(northyork_fsinfo)

Pickling info from Foursquare:

In [24]:
#pickling
with open("northyork_fsinfo.txt", "wb") as fp:
    pickle.dump(northyork_fsinfo, fp)

#unpickling
with open("northyork_fsinfo.txt", "rb") as fp:
    northyork_fsinfo = pickle.load(fp)
    
##northyork_fsinfo

Preparing data in the North York neighborhoods for analysis:

In [25]:
#define function to obtain venues from the local North York info already obtained from FourSquare
def get_venue(fs_info):
    
    #initialization
    result = pd.DataFrame(columns = [
        'Postal Code', 
        'Neighborhood', 
        'Neighborhood Latitude', 
        'Neighborhood Longitude', 
        'Venue', 
        'Venue Summary', 
        'Venue Category', 
        'Distance'])
    
    for neighborhood_dict in fs_info:
        postal_code = neighborhood_dict['Postal Code']; 
        neigh = neighborhood_dict['Neighborhood(s)'];
        lat = neighborhood_dict['Latitude']; 
        lng = neighborhood_dict['Longitude'];
        
        for venue_dict in neighborhood_dict['Crawling_result']:
            name = venue_dict['venue']['name']
            summary = venue_dict['reasons']['items'][0]['summary']
            category = venue_dict['venue']['categories'][0]['name']
            distance = venue_dict['venue']['location']['distance']
            result = result.append({
                'Postal Code': postal_code, 
                'Neighborhood': neigh, 
                'Neighborhood Latitude': lat, 
                'Neighborhood Longitude':lng,
                'Venue': name, 
                'Venue Summary': summary, 
                'Venue Category': category, 
                'Distance': distance}, 
                ignore_index = True)
    
    return(result)

#execute the above function to get venues
northyork_venue = get_venue(northyork_fsinfo)
northyork_venue.size

8184

In [26]:
#preview head
northyork_venue.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M3A,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,This spot is popular,Caribbean Restaurant,833
1,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,This spot is popular,Park,245
2,M3A,Parkwoods,43.753259,-79.329656,Tim Hortons,This spot is popular,Café,866
3,M3A,Parkwoods,43.753259,-79.329656,A&W Canada,This spot is popular,Fast Food Restaurant,852
4,M3A,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,This spot is popular,Grocery Store,882


In [24]:
#preview tail
northyork_venue.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
1020,M2R,Willowdale West,43.782736,-79.442259,Antibes Park,This spot is popular,Park,673
1021,M2R,Willowdale West,43.782736,-79.442259,Northview Variety,This spot is popular,Convenience Store,842
1022,M2R,Willowdale West,43.782736,-79.442259,Belle Restaurant,This spot is popular,Eastern European Restaurant,936
1023,M2R,Willowdale West,43.782736,-79.442259,Happy Bagel Bakery,This spot is popular,Bakery,972
1024,M2R,Willowdale West,43.782736,-79.442259,TTC Bus 36 Finch West,This spot is popular,Bus Line,989


Save a clean version of northyork_venue dataframe and re-open:

In [27]:
#saving
northyork_venue.to_csv('northyork_venue.csv')

#reloading
northyork_venue = pd.read_csv('northyork_venue.csv')

Understand neighborhood information of North York:

In [28]:
#show neighborhood names
neighborhood_info = list(northyork_venue['Neighborhood'].unique())
neighborhood_info

['Parkwoods',
 'Victoria Village',
 'Lawrence Heights',
 'Lawrence Manor',
 'Don Mills Nort',
 'Glencair',
 'Flemingdon Park',
 'Don Mills Sout',
 'Hillcrest Village',
 'Bathurst Manor',
 'Downsview Nort',
 'Wilson Heights',
 'Fairvie',
 'Henry Farm',
 'Oriol',
 'Northwood Park',
 'York University',
 'Bayview Village',
 'CFB Toronto',
 'Downsview Eas',
 'Silver Hill',
 'York Mills',
 'Downsview West',
 'Downsview',
 'North Par',
 'Upwood Par',
 'Humber Summit',
 'Newtonbrook',
 'Willowdale',
 'Downsview Centra',
 'Bedford Park',
 'Lawrence Manor Eas',
 'Emery',
 'Humberlea',
 'Willowdale Sout',
 'Downsview Northwes',
 'York Mills Wes',
 'Willowdale West']

In [29]:
#show number of neighborhood
len(neighborhood_info)

38

In [30]:
#show unique category names
list(northyork_venue['Venue Category'].unique())

['Caribbean Restaurant',
 'Park',
 'Café',
 'Fast Food Restaurant',
 'Grocery Store',
 'Supermarket',
 'Pharmacy',
 'Fish & Chips Shop',
 'Food & Drink Shop',
 'Pizza Place',
 'Road',
 'Bus Stop',
 'Train Station',
 'Discount Store',
 'Laundry Service',
 'Chinese Restaurant',
 'Coffee Shop',
 'Convenience Store',
 'Shopping Mall',
 'Skating Rink',
 'Tennis Court',
 'Cosmetics Shop',
 'Shop & Service',
 'Hockey Arena',
 'Portuguese Restaurant',
 'Intersection',
 'Sporting Goods Shop',
 "Men's Store",
 'Lounge',
 'Golf Course',
 'Athletics & Sports',
 'Gym / Fitness Center',
 'Boutique',
 'Vietnamese Restaurant',
 'Sushi Restaurant',
 'Furniture / Home Store',
 'Deli / Bodega',
 'Dessert Shop',
 'Burger Joint',
 'Fried Chicken Joint',
 'Greek Restaurant',
 'Restaurant',
 'Bowling Alley',
 'Clothing Store',
 'Accessories Store',
 'Bank',
 'Seafood Restaurant',
 'Pet Store',
 'Gym',
 'Event Space',
 'Miscellaneous Shop',
 'Cheese Shop',
 'Fraternity House',
 "Women's Store",
 'Sandwich Pla

In [32]:
#show number of unique category
len(northyork_venue['Venue Category'].unique())

156

Perform "one-hot encoding" into every category

In [33]:
# one hot encoding
northyork_onehot = pd.get_dummies(
    data = northyork_venue, 
    drop_first = False,
    prefix = "", 
    prefix_sep = "", 
    columns = ['Venue Category']
)

In [34]:
#preview northyork_onehot head
northyork_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bowling Alley,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Frame Store,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health Food Store,History Museum,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Optical Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,0,M3A,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,This spot is popular,833,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,This spot is popular,245,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M3A,Parkwoods,43.753259,-79.329656,Tim Hortons,This spot is popular,866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M3A,Parkwoods,43.753259,-79.329656,A&W Canada,This spot is popular,852,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M3A,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,This spot is p

In [35]:
#preview northyork_onehot tail
northyork_onehot.tail()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bowling Alley,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Frame Store,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health Food Store,History Museum,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Optical Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
1018,1018,M2R,Willowdale West,43.782736,-79.442259,Antibes Park,This spot is popular,673,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1019,1019,M2R,Willowdale West,43.782736,-79.442259,Northview Variety,This spot is popular,842,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1020,1020,M2R,Willowdale West,43.782736,-79.442259,Belle Restaurant,This spot is popular,936,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1021,1021,M2R,Willowdale West,43.782736,-79.442259,Happy Bagel Bakery,This spot is popular,972,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1022,1022,M

In [36]:
#assign feature list based on category
feature_list = [
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',

 'Caribbean Restaurant',
 'Café',
 'Fast Food Restaurant',
 'Fish & Chips Shop',
 'Food & Drink Shop',
 'Pizza Place',
 'Chinese Restaurant',
 'Portuguese Restaurant',
 'Vietnamese Restaurant',
 'Sushi Restaurant',
 'Burger Joint',
 'Fried Chicken Joint',
 'Greek Restaurant',
 'Restaurant',
 'Seafood Restaurant',
 'Sandwich Place',
 'Korean Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Asian Restaurant',
 'Salad Place',
 'Thai Restaurant',
 'Diner',
 'Breakfast Spot',
 'Cafeteria',
 'Latin American Restaurant',
 'Mediterranean Restaurant',
 'Middle Eastern Restaurant',
 'Dim Sum Restaurant',
 'American Restaurant',
 'Falafel Restaurant',
 'Turkish Restaurant',
 'Empanada Restaurant',
 'Hot Dog Joint',
 'Indian Restaurant',
 'Ramen Restaurant',
 'Comfort Food Restaurant',
 'Wings Joint',
 'Steakhouse',
 'Indonesian Restaurant',
 'Burrito Place',
 'Snack Place',
 'French Restaurant',
 'Eastern European Restaurant'
]

Group the feature list by neighborhood:

In [37]:
#drop category not in the feature_list
northyork_onehot = northyork_onehot[feature_list].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby('Neighborhood').sum()

#preview northyork_onehot head
northyork_onehot.head()

,Caribbean Restaurant,Café,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Pizza Place,Chinese Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Sushi Restaurant,Burger Joint,Fried Chicken Joint,Greek Restaurant,Restaurant,Seafood Restaurant,Sandwich Place,Korean Restaurant,Italian Restaurant,Japanese Restaurant,Asian Restaurant,Salad Place,Thai Restaurant,Diner,Breakfast Spot,Cafeteria,Latin American Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Dim Sum Restaurant,American Restaurant,Falafel Restaurant,Turkish Restaurant,Empanada Restaurant,Hot Dog Joint,Indian Restaurant,Ramen Restaurant,Comfort Food Restaurant,Wings Joint,Steakhouse,Indonesian Restaurant,Burrito Place,Snack Place,French Restaurant,Eastern European Restaurant
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Bathurst Manor,0,0,1,0,0,2,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bayview Village,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bedford Park,0,1,3,0,0,1,0,0,0,1,0,0,1,1,0,1,0,3,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0
CFB Toronto,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
Don Mills Nort,1,1,0,0,0,2,0,0,0,0,2,0,1,1,0,0,0,0,3,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
northyork_onehot.shape

(38, 44)

In [39]:
#initialization
feat_name_list = list(northyork_onehot.columns)

#aggregate all restaurants into one column
restaurant_list = []
for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
northyork_onehot['Total Restaurants'] = northyork_onehot[restaurant_list].sum(axis = 1)
northyork_onehot = northyork_onehot.drop(columns = restaurant_list)

#aggregate all joints into one column
feat_name_list = list(northyork_onehot.columns)
joint_list = []
for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
northyork_onehot['Total Joints'] = northyork_onehot[joint_list].sum(axis = 1)
northyork_onehot = northyork_onehot.drop(columns = joint_list)

#preview northyork_onehot head
northyork_onehot.head()

,Café,Fish & Chips Shop,Food & Drink Shop,Pizza Place,Sandwich Place,Salad Place,Diner,Breakfast Spot,Cafeteria,Steakhouse,Burrito Place,Snack Place,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,,,,
Bathurst Manor,0,0,0,2,1,0,1,0,0,0,0,0,5,1
Bayview Village,1,0,0,0,0,0,0,0,0,0,0,0,4,0
Bedford Park,1,0,0,1,1,0,0,0,0,0,0,0,14,1
CFB Toronto,1,0,0,1,1,0,0,0,0,0,0,0,5,0
Don Mills Nort,1,0,0,2,0,1,1,1,1,0,0,0,8,2


Run machine learning using k-means to cluster neighborhoods into 7 clusters:

In [40]:
#execute k-means algorithm
kmeans = KMeans(n_clusters = 7, random_state = 0).fit(northyork_onehot)

Show centers of each cluster and sort in descending order:

In [41]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = northyork_onehot.columns
means_df.index = ['Group 1','Group 2','Group 3','Group 4','Group 5','Group 6','Group 7']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Café,Fish & Chips Shop,Food & Drink Shop,Pizza Place,Sandwich Place,Salad Place,Diner,Breakfast Spot,Cafeteria,Steakhouse,Burrito Place,Snack Place,Total Restaurants,Total Joints,Total Sum
Group 2,3.000000,0.000000,0.000000,5.000000,4.000000,0.000000,0.000000e+00,0.000000,0.000000,1.0,1.0,0.000000,37.000000,3.000000e+00,54.000000
Group 4,0.333333,0.000000,0.000000,0.333333,1.000000,0.000000,-2.775558e-17,0.000000,0.000000,0.0,0.0,0.000000,13.666667,1.666667e+00,17.000000
Group 1,1.166667,0.000000,0.000000,1.500000,1.333333,0.166667,1.666667e-01,0.166667,0.166667,0.0,0.0,0.000000,8.666667,2.000000e+00,15.333333
Group 7,0.000000,0.000000,0.000000,2.666667,1.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.0,0.000000,11.000000,0.000000e+00,14.666667
Group 5,0.333333,0.000000,0.000000,1.666667,1.000000,0.000000,5.000000e-01,0.000000,0.000000,0.0,0.0,0.166667,5.166667,6.666667e-01,9.500000
Group 6,0.222222,0.111111,0.111111,0.666667,0.444444,0.000000,2.222222e-01,0.000000,0.000000,0.0,0.0,0.000000,2.888889,-1.110223e-16,4.666667
Group 3,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,-2.775558e-17,0.000000,0.000000,0.0,0.0,0.000000,0.428571,-1.110223e-16,0.714286


**Ranking:**  
**1. Group 2**  
**2. Group 4**  
**3. Group 1**  
**4. Group 7**  
**5. Group 5**  
**6. Group 6**  
**7. Group 3**  

Insert labels to each group:

In [42]:
neighborhood_summary = pd.DataFrame([northyork_onehot.index, 1 + kmeans.labels_]).T
neighborhood_summary.columns = ['Neighborhood', 'Group']
neighborhood_summary

,Neighborhood,Group
0,Bathurst Manor,5
1,Bayview Village,6
2,Bedford Park,4
3,CFB Toronto,5
4,Don Mills Nort,1
5,Don Mills Sout,4
6,Downsview,6
7,Downsview Centra,6
8,Downsview Eas,5
9,Downsview Nort,5


**Rank 1: Group 2:**

In [133]:
neighborhood_summary[neighborhood_summary['Group'] == 2]

,Neighborhood,Group
32,Willowdale Sout,2


**Rank 2: Group 4:**

In [134]:
neighborhood_summary[neighborhood_summary['Group'] == 4]

,Neighborhood,Group
2,Bedford Park,4
5,Don Mills Sout,4
14,Flemingdon Park,4
20,Lawrence Heights,4
21,Lawrence Manor,4
22,Lawrence Manor Eas,4


**Rank 3: Group 1**

In [135]:
neighborhood_summary[neighborhood_summary['Group'] == 1]

,Neighborhood,Group
4,Don Mills Nort,1
13,Fairvie,1
16,Henry Farm,1
23,Newtonbrook,1
26,Oriol,1
31,Willowdale,1


**Rank 4: Group 7**

In [44]:
neighborhood_summary[neighborhood_summary['Group'] == 7]

,Neighborhood,Group
15,Glencair,7
25,Northwood Park,7
37,York University,7


**Rank 5: Group 5**

In [45]:
neighborhood_summary[neighborhood_summary['Group'] == 5]

,Neighborhood,Group
0,Bathurst Manor,5
3,CFB Toronto,5
8,Downsview Eas,5
9,Downsview Nort,5
10,Downsview Northwes,5
34,Wilson Heights,5


**Rank 6: Group 6**

In [46]:
neighborhood_summary[neighborhood_summary['Group'] == 6]

,Neighborhood,Group
1,Bayview Village,6
6,Downsview,6
7,Downsview Centra,6
17,Hillcrest Village,6
18,Humber Summit,6
24,North Par,6
27,Parkwoods,6
29,Upwood Par,6
36,York Mills Wes,6


**Rank 7: Group 3**

In [47]:
neighborhood_summary[neighborhood_summary['Group'] == 3]

,Neighborhood,Group
11,Downsview West,3
12,Emery,3
19,Humberlea,3
28,Silver Hill,3
30,Victoria Village,3
33,Willowdale West,3
35,York Mills,3
